## Experiment on LLM model performance

Setup the git repo and the models, download kokoro files, links in the readme.md and create a folder called kokoro in in the MSGAI-AI-Podcast-Generator

In [ ]:
branch_name = "few-shot-examples"
repo_url = "https://github.com/rkaushik97/MSGAI-AI-Podcast-Generator.git"

!git clone -b {branch_name} --single-branch {repo_url}

In [ ]:
from google.colab import userdata
from huggingface_hub import login
# setting key in secrets google colab
hf_token = userdata.get('HUGGINGFACE_API_KEY')
login(hf_token, add_to_git_credential=False)

In [ ]:
%cd MSGAI-AI-Podcast-Generator/podcast-generator

In [ ]:
# !pip install -U piper-tts kokoro-onnx

## Experiment

In [ ]:
from podcast_pipeline.llm_generator import LLMScriptGenerator

LLM_MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"
prompt_template = "podcast_script_v1"

llm = LLMScriptGenerator(model_id=LLM_MODEL_ID)
# tts = AdaptiveTTSSynthesizer()

In [ ]:
# a subset of 10 topics taken from topics_batch_1, that are used to test the model quality
podcast_topics = [
    "The Simulation Theory: Are we living in a video game?",
    "De-extinction: bringing the Woolly Mammoth back",
    "Artificial General Intelligence vs. Human Intuition",
    "The psychology behind why we believe conspiracy theories",
    "The future of commercial space tourism",
    "Theoretical physics of time travel",
    "Brain-Computer Interfaces: Merging mind and machine",
    "The Fermi Paradox: Where is everybody?",
    "The search for the Lost City of Atlantis",
    "Cryptozoology: The hunt for Bigfoot and Nessie"
]

In [ ]:
experiment_params = {
    "few_shot_examples_nr": [0,5], #[0, 1, 3, 5],
    "max_token_len": [256], #[64, 128,256, 512, 1024],
    "temperature": [0.8],  # encourage originality
    "num_podcasts": 10,
    "topics": podcast_topics,
    "cold_start": [True]
}

In [ ]:
import torch
import time
import json

def run_experiment(model_id, prompt_template, topics, params, output_dir):
    """
    Runs inference across topics while measuring generation latency and token counts.
    Supports cold-start / warm-start per topic.
    """
    results = []

    # Iterate over cold_start modes
    for cold in params.get("cold_start", [True]):
        print(f"\n=== Experiment mode: cold_start={cold} ===")

        # Initialize once if warm start
        llm = None
        if not cold:
            print("Warm-start mode: initializing model once and performing dummy inference...")
            llm = LLMScriptGenerator(model_id=model_id)
            _ = llm.generate(
                "Warm-up dummy question",
                prompt_template,
                max_new_tokens=256,
                temperature=0.1
            )
            print("Warm-up completed.")

        # -----------------------------------------------------------
        # MAIN LOOP OVER TOPICS
        # -----------------------------------------------------------
        for topic in topics[: params["num_podcasts"]]:
            print(f"\nGenerating podcast script for topic: {topic}")

            for few_shot_nr in params["few_shot_examples_nr"]:
                for max_len in params["max_token_len"]:
                    temperature = params["temperature"][0]

                    # ---------------------------------------------------
                    # Cold-start handling: reload model every topic
                    # ---------------------------------------------------
                    if cold:
                        print(" → Cold-start: initializing model...")
                        start_init = time.time()
                        llm = LLMScriptGenerator(model_id=model_id)
                        end_init = time.time()
                        model_load_time = end_init - start_init
                    else:
                        model_load_time = 0.0  # no reload for warm start

                    # ---------------------------------------------------
                    # Actual inference
                    # ---------------------------------------------------
                    start_time = time.time()
                    script, _ = llm.generate(
                        topic,
                        prompt_template,
                        few_shot_examples_nr=few_shot_nr,
                        max_new_tokens=max_len,
                        temperature=temperature
                    )
                    end_time = time.time()
                    inference_time = end_time - start_time

                    # ---------------------------------------------------
                    # Token stats (optional)
                    # ---------------------------------------------------
                    try:
                        messages = llm._create_prompt(topic, few_shot_examples_nr=few_shot_nr)
                        # Concatenate all message contents into a single string
                        prompt_text = "\n".join([m["content"] for m in messages])

                        input_tokens = llm.count_tokens(prompt_text)

                        # Convert metadata to string
                        metadata_text = "\n".join([f"{k}: {v}" for k, v in script['metadata'].items()])

                        # Reconstruct the generated text
                        reconstructed_output = f"---METADATA---\n{metadata_text}\n---DIALOGUE---\n{script['dialogue']}"

                        # Count tokens
                        output_tokens = llm.count_tokens(reconstructed_output)
                    except Exception as e:
                        print(e)
                        print(f"Error computing tokens for topic: {topic}, few_shot={few_shot_nr}")
                        input_tokens = None
                        output_tokens = None

                    # ---------------------------------------------------
                    # Store results
                    # ---------------------------------------------------
                    ## Would have been better to save also the original prompt input
                    results.append({
                        "topic": topic,
                        "few_shot_nr": few_shot_nr,
                        "max_tokens": max_len,
                        "temperature": temperature,
                        "cold_start": cold,
                        "model_load_seconds": model_load_time,
                        "inference_seconds": inference_time,
                        "total_time_seconds": model_load_time + inference_time,
                        "output_text": script,
                        "prompt_token_count": input_tokens,
                        "output_token_count": output_tokens,
                        "total_tokens": (
                            input_tokens + output_tokens
                            if input_tokens and output_tokens else None
                        )
                    })

                    print(
                        f"  → few-shot={few_shot_nr}, max_len={max_len}, "
                        f"cold_start={cold}, load={model_load_time:.2f}s, infer={inference_time:.2f}s"
                    )

                    # Cleanup model for next cold start if needed
                    if cold:
                        del llm
                        torch.cuda.empty_cache()

    with open(output_dir, 'w') as outfile:
      json.dump(results, outfile)
    return results

results = run_experiment(LLM_MODEL_ID, prompt_template, podcast_topics, experiment_params, 'full_exp.json')

In [ ]:
import time

start_llm_gen = time.time()
script, scores = llm.generate(podcast_topics[0], prompt_template)
end_llm_gen = time.time()

# start_tts_gen = time.time()
# tts.synthesize(script, 'test.wav')
# end_tts_gen = time.time()

print(end_llm_gen - start_llm_gen)
# print(end_tts_gen - start_tts_gen)

## Few shot examples creation
Generate new topics using GPT-5 mini



In [ ]:
few_shot_examples = [
  {
    "topic": "The rise of AI-generated art and its impact on creativity",
    "script": ""
  },
  {
    "topic": "Forgotten inventions that were ahead of their time",
    "script": ""
  },
  {
    "topic": "The science of dreams: decoding what our subconscious is telling us",
    "script": ""
  },
  {
    "topic": "Virtual reality therapy: treating phobias, PTSD, and social anxiety",
    "script": ""
  },
  {
    "topic": "The mysteries of human memory: why we forget and misremember",
    "script": ""
  },
  {
    "topic": "Ancient astronomical knowledge and lost observatories",
    "script": ""
  },
  {
    "topic": "The future of deepfake technology and digital identity",
    "script": ""
  },
  {
    "topic": "Microbiomes and how gut bacteria influence behavior",
    "script": ""
  },
  {
    "topic": "The untold history of secret societies and hidden power structures",
    "script": ""
  },
  {
    "topic": "Exploring the psychology of extreme sports and risk-taking behavior",
    "script": ""
  }
]

In [ ]:
# create the prompt that then I have passed to gpt5-mini
# for ex in few_shot_examples:
#   ex['prompt'] = llm._create_prompt(ex['topic'])

In [ ]:
# import json

# with open('few_shot_examples.json', 'w') as file:
#   json.dump(few_shot_examples, file)

# with open('few_shot_examples.txt', 'w') as file:
#   for ex in few_shot_examples:
#     file.write(ex['prompt'][0]['content'])
#     file.write('\n')
#     file.write('#'*100)

Prompt with few shot examples generated by GPT5-mini

In [ ]:
llm._create_prompt(few_shot_examples[0]['topic'], few_shot_examples_nr=1)

## LLM as judge
Model page: https://huggingface.co/meta-llama/Meta-Llama-3-70B

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/meta-llama/Meta-Llama-3-70B)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# LLM_JUDGE_ID = "meta-llama/Meta-Llama-3-70B" (resources were insufficient)
# LLM_JUDGE_ID = "meta-llama/Meta-Llama-3.1-34B-Instruct" #(not available)
# LLM_JUDGE_ID = "mistralai/mixtral-8x22b-instruct-v01" #(not available)

LLM_JUDGE_ID = "mistralai/mistral-7b-instruct-v0.3"
# LLM_JUDGE_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct" #(the model itself)


LLM_JUDGE_TASK = "text-generation"

In [ ]:
from podcast_pipeline.llm_generator import LLMScriptGenerator

llm_judge = LLMScriptGenerator(model_id=LLM_JUDGE_ID, model_task=LLM_JUDGE_TASK)
# llm_judge = LLMScriptGenerator()

In [ ]:
import json
import re
from podcast_pipeline.llm_generator import LLMScriptGenerator

def evaluate_result(result, model_id, template_key):
  # reconstruct the original prompt
  original_prompt = llm_judge._create_prompt(result['topic'])[0]['content']

  # topic of the request and generated script {topic: , metadata: , dialogue: }
  original_topic = result['topic']
  generated_script = result["output_text"]

  # reconstruct the string of the original output
  output = LLMScriptGenerator.reconstruct_llm_output(generated_script['metadata'], generated_script['dialogue'])

  # format the prompt for the judge (v2 include also the original prompt -> compliance score)
  system_prompt, user_query = llm_judge._prompt_manager.format_judge_prompt(
                  template_key,
                  original_topic=original_topic,
                  generated_script=output,
                  original_prompt=original_prompt
              )

  if model_id not in ['meta-llama/Meta-Llama-3.1-8B-Instruct','mistralai/mistral-7b-instruct-v0.3']:
    print("The model is not integrated, set input prompt creation and output extraction :)")
    return None
  elif model_id == 'mistralai/mistral-7b-instruct-v0.3':
    # mixtral formatting
    prompt_input = (
        "<s>[INST] "
        f"{system_prompt}\n\n{user_query} "
        "[/INST] "
    )
  elif model_id == 'meta-llama/Meta-Llama-3.1-8B-Instruct':
    # prompt formatted as a conversation (8b LLAMA)
    prompt_input = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]


  judge_output = llm_judge._pipeline(
      prompt_input,
      max_new_tokens=256,
      do_sample=False,
      temperature=0.1,
      pad_token_id=llm_judge._pipeline.tokenizer.eos_token_id,
  )

  if model_id == 'mistralai/mistral-7b-instruct-v0.3':
    # Extract generated text
    judge_text = judge_output[0]["generated_text"]

  elif model_id == 'meta-llama/Meta-Llama-3.1-8B-Instruct':
    # Extract the generated content (assuming it's the last message content)
    judge_text = judge_output[0]["generated_text"][-1]['content']

  else:
    print(f'Define how the the output should be extracted for {model_id}')
    return None

  # # Attempt to extract JSON from the output
  json_match = re.search(r"\{.*\}", judge_text, re.DOTALL)
  if json_match:
      evaluation = json.loads(json_match.group(0))
  else:
      # fallback if JSON not found
      evaluation = {"relevance_score": None, "coherence_score": None, "compliance_score": None}

  return evaluation

In [ ]:
import json
from tqdm import tqdm

# load exp results
results_path = '../notebooks/llm_judge/full_exp_no_scores.json'
# results_path = 'full_exp.json'
with open(results_path) as f:
    results = json.load(f)

# template_judge_key = "llm_judge_v2" #(granularity 1-5)
template_judge_key = "llm_judge_v3" #(granularity 1-10)

# llama 70b expect a chat template
for entry in tqdm(results):
    judge_output = evaluate_result(entry, LLM_JUDGE_ID, template_judge_key)
    entry['relevance_score'] = judge_output.get("relevance_score", None)
    entry["coherence_score"] = judge_output.get("coherence_score", None)
    entry["compliance_score"] = judge_output.get("compliance_score", None)

# Optionally save back to JSON
with open("results_with_scores.json", "w") as f:
    json.dump(results, f, indent=2)


## Test Judge on few shot examples (originals and with errors)

In [ ]:
def evaluate_few_shot_example(few_shot_example, model_id, template_key):
  # reconstruct the original prompt
  original_prompt = few_shot_example['prompt'][0]['content']

  # topic of the request and generated script {topic: , metadata: , dialogue: }
  original_topic = few_shot_example['topic']
  output = few_shot_example["script"]

  # format the prompt for the judge (v2 include also the original prompt -> compliance score)
  system_prompt, user_query = llm_judge._prompt_manager.format_judge_prompt(
                  template_key,
                  original_topic=original_topic,
                  generated_script=output,
                  original_prompt=original_prompt
              )

  if model_id not in ['meta-llama/Meta-Llama-3.1-8B-Instruct','mistralai/mistral-7b-instruct-v0.3']:
    print("The model is not integrated, set input prompt creation and output extraction :)")
    return None
  elif model_id == 'mistralai/mistral-7b-instruct-v0.3':
    # mixtral formatting
    prompt_input = (
        "<s>[INST] "
        f"{system_prompt}\n\n{user_query} "
        "[/INST] "
    )
  elif model_id == 'meta-llama/Meta-Llama-3.1-8B-Instruct':
    # prompt formatted as a conversation (8b LLAMA)
    prompt_input = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]

  judge_output = llm_judge._pipeline(
      prompt_input,
      max_new_tokens=256,
      do_sample=False,
      temperature=0.1,
      pad_token_id=llm_judge._pipeline.tokenizer.eos_token_id,
  )

  if model_id == 'mistralai/mistral-7b-instruct-v0.3':
    # Extract generated text
    judge_text = judge_output[0]["generated_text"]

  elif model_id == 'meta-llama/Meta-Llama-3.1-8B-Instruct':
    # Extract the generated content (assuming it's the last message content)
    judge_text = judge_output[0]["generated_text"][-1]['content']

  else:
    print(f'Define how the the output should be extracted for {model_id}')
    return None

  # # Attempt to extract JSON from the output
  json_match = re.search(r"\{.*\}", judge_text, re.DOTALL)
  if json_match:
      evaluation = json.loads(json_match.group(0))
  else:
      # fallback if JSON not found
      evaluation = {"relevance_score": None, "coherence_score": None, "compliance_score": None}

  return evaluation

In [ ]:
import json
from tqdm import tqdm
import re

# load exp results
# few_shot_examples_path = 'input/few_shot_examples_responses.json'
few_shot_examples_path = 'input/few_shot_examples_responses_with_errors.json'

with open(few_shot_examples_path) as f:
    few_shot_examples = json.load(f)

# template_judge_key = "llm_judge_v2" #(granularity 1-5)
template_judge_key = "llm_judge_v3" #(granularity 1-10)

# llama 70b expect a chat template
for entry in tqdm(few_shot_examples):
    judge_output = evaluate_few_shot_example(entry, LLM_JUDGE_ID, template_judge_key)
    entry['relevance_score'] = judge_output.get("relevance_score", None)
    entry["coherence_score"] = judge_output.get("coherence_score", None)
    entry["compliance_score"] = judge_output.get("compliance_score", None)

# Optionally save back to JSON
with open("few_shot_examples_with_scores.json", "w") as f:
    json.dump(few_shot_examples, f, indent=2)